In [1]:
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, callback
import folium

In [2]:
crimes = pd.read_csv('data/data.csv')
crimes.head()

,Unnamed: 0,Date,Time of Day,Offense,Method,Violence,Block,Police District,PSA,Ward,ANC,Longitude,Latitude,Year,Month,Day,Hour,Minute
0,0,2008-08-31 20:47:00,EVENING,THEFT/OTHER,OTHER,Non-Violent,3500 - 3599 BLOCK OF R STREET NW,2.0,206.0,2,2E,-77.070209,38.913633,2008,8,31,20,47
1,1,2008-09-01 00:45:00,MIDNIGHT,MOTOR VEHICLE THEFT,OTHER,Non-Violent,2000 - 2015 BLOCK OF O STREET NW,2.0,208.0,2,2B,-77.045323,38.908564,2008,9,1,0,45
2,2,2008-09-01 03:00:00,MIDNIGHT,THEFT/OTHER,OTHER,Non-Violent,700 - 799 BLOCK OF 11TH STREET NW,1.0,101.0,2,2C,-77.027045,38.899057,2008,9,1,3,0
3,3,2008-09-09 07:46:00,DAY,THEFT/OTHER,OTHER,Non-Violent,1700 - 1799 BLOCK OF P STREET NW,2.0,208.0,2,2B,-77.040089,38.909638,2008,9,9,7,46
4,4,2008-08-24 20:00:00,EVENING,MOTOR VEHICLE THEFT,OTHER,Non-Violent,500 - 599 BLOCK OF INDIANA AVENUE NW,1.0,102.0,2,2C,-77.019377,38.894573,2008,8,24,20,0


In [3]:
# dc center point for folium
dc_lat = (crimes['Latitude'].min() + crimes['Latitude'].max())/2
dc_long = (crimes['Longitude'].min() + crimes['Longitude'].max())/2
dc_center = (dc_lat, dc_long)

# create a new map object
map = folium.Map(location=dc_center, zoom_start=12)

# save our map to an interactive html file
map.save('initial_dc_crimes_map.html')

In [4]:
# load stylesheet
stylesheets = ['https://codepen.io/chriddyp/pen/dZVMbK.css']

# initialize app
app = Dash(__name__, external_stylesheets=stylesheets)
server = app.server

In [5]:
app.layout = html.Div([
    # title row
    html.Div(
        html.H1(
            "Mapping of Washington D.C. Crime History",
            style = {"color": "light grey", "font-family": "serif"}),
        className = "row"
    ),

    # graph row
    html.Div(
        id = 'map graph', 
        children = html.Iframe(
            id='initial map graph',
            srcDoc = open('initial_dc_crimes_map.html','r').read(),
            style={'width': '100%', 'height':'400px','display': 'inline-block'},
        )
    ),
    
    # checklist row
    html.Div(
        dcc.Checklist(
            crimes['Offense'].unique(),
            crimes['Offense'].unique(),
            inline=True,
            style={'color': 'Gold', 'font-size': 15},
            id = 'offense',
        )
    ),

    # slider, and dropdowns row
    html.Div([

        # timeframe slider
        html.Div(
            children = [
                html.Label(
                    'Year',
                    style={"color": "light grey"}
                ),
                dcc.Slider(
                    min = crimes['Year'].min(), 
                    max = crimes['Year'].max(), 
                    step = 1.0, 
                    marks={i: '{}'.format(i) for i in range(crimes['Year'].min(), crimes['Year'].max())},
                    id = 'year',
                    tooltip = {"placement": "bottom", "always_visible": True},
                    included = False,
                    value = crimes['Year'].min(),
                ),
            ],
            className = "six columns"
        ),

        # wards dropdown
        html.Div(
            children = [
                html.Label(
                    'Wards',
                    style = {"color": "light grey"}
                ),
                dcc.Dropdown(
                    id = 'wards',
                    options = [{"label": x, "value": x} for x in sorted(crimes["Ward"].unique())],
                    multi = True,
                    value = [],
                    placeholder = 'Select Wards',
                )
            ], 
            className="five columns"
        ),

        # time of day dropdown
        html.Div(
            children = [
                html.Label(
                    'Time of Day',
                    style = {"color": "light grey"}
                ),
                dcc.Dropdown(
                    id = 'time of day',
                    options = crimes['Time of Day'].unique(),
                    value = 'DAY'
                )
            ], 
            className="one column"
        ),
    ]),
])

# define callbacks
@app.callback(
    Output('map graph', 'children'),
    Input('year', 'value'),
    Input('offense', 'value'),
    Input('wards', 'value'),
    Input('time of day', 'value'))
def update_figure(selected_year, selected_offense, selected_wards, selected_time_of_day):
    # create a filtered dataset
    filtered_df = crimes.loc[(crimes['Year'] == selected_year) & 
                             (crimes['Ward'].isin(selected_wards)) &
                             (crimes['Time of Day'] == selected_time_of_day) &
                             (crimes['Offense'].isin(selected_offense))]

    # dc center point for folium
    dc_lat = (crimes['Latitude'].min() + crimes['Latitude'].max())/2
    dc_long = (crimes['Longitude'].min() + crimes['Longitude'].max())/2
    dc_center = (dc_lat, dc_long)

    # create a new map object
    map = folium.Map(location=dc_center, zoom_start=12)

    # add crimes
    for i in range(len(filtered_df)):
        folium.Circle(
            location=[filtered_df.iloc[i]['Latitude'], filtered_df.iloc[i]['Longitude']],
            radius=16,
        ).add_to(map)

    # save our map to an interactive html file
    map_html = map.get_root().render()

    return html.Iframe(
            srcDoc = map_html,
            style={'width': '100%', 'height':'400px','display': 'inline-block'},
        )

In [6]:
if __name__ == '__main__':
    app.run(debug=True)